Part A: For 4 digit MPIN
1. From Demographic Data
2. From Commonly Used Patterns

In [1]:
import json
import os
from datetime import datetime
from itertools import permutations, combinations

def load_common_mpins():
    common_mpins = set()
    digits = '0123456789'
    # All identical digits
    for d in digits:
        common_mpins.add(d * 4)
    # Palindromes ABBA
    for a in digits:
        for b in digits:
            common_mpins.add(f"{a}{b}{b}{a}")
    # Alternating (ABAB, BABA)
    for a in digits:
        for b in digits:
            common_mpins.add(f"{a}{b}{a}{b}")
            common_mpins.add(f"{b}{a}{b}{a}")
    # Double blocks (AABB, BBAA)
    for a in digits:
        for b in digits:
            common_mpins.add(f"{a}{a}{b}{b}")
            common_mpins.add(f"{b}{b}{a}{a}")
    # Simple consecutive ascending/descending without wrap
    for start in range(0, 7):
        common_mpins.add(''.join(str(start + i) for i in range(4)))
    for start in range(9, 2, -1):
        common_mpins.add(''.join(str(start - i) for i in range(4)))
    # All arithmetic progressions of 4 digits modulus 10
    for step in range(1, 5):
        for start in range(0, 10):
            seq = [str((start + i * step) % 10) for i in range(4)]
            common_mpins.add("".join(seq))
            if len(set(seq)) == 4:
                for p in set(permutations(seq, 4)):
                    common_mpins.add("".join(p))
    return common_mpins

def is_valid_mpin(mpin):
    return isinstance(mpin, str) and mpin.isdigit() and len(mpin) == 4

def is_valid_phone_number(phone_number):
    return isinstance(phone_number, str) and phone_number.isdigit() and len(phone_number) == 10

def is_valid_pincode(pincode):
    return isinstance(pincode, str) and pincode.isdigit() and len(pincode) == 6

def is_valid_date(date, is_dob=True):
    if not date or '-' not in date:
        return False
    try:
        day, month, year = map(int, date.split('-'))
        if not (1 <= day <= 31 and 1 <= month <= 12 and 1900 <= year <= 2025):
            return False
        if month in [4, 6, 9, 11] and day > 30:
            return False
        if month == 2:
            is_leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
            if (is_leap and day > 29) or (not is_leap and day > 28):
                return False
        input_date = datetime(year, month, day)
        current_date = datetime(2025, 6, 30)
        if is_dob and input_date > current_date:
            return False
        if not is_dob and (input_date > current_date or year < 1900):
            return False
        return True
    except (ValueError, TypeError):
        return False

def prompt_valid_date(prompt, is_dob=True):
    while True:
        date = input(prompt).strip()
        if not date:
            return ""
        if is_valid_date(date, is_dob):
            return date
        error_msg = (
            "Invalid date. Use DD-MM-YYYY format (e.g., 02-01-1998). "
            f"{'DOB must not be in the future.' if is_dob else 'Anniversary must be between 1900 and June 30, 2025.'} "
            "Try again or press Enter to skip."
        )
        print(error_msg)

def load_demographics(phone_number):
    if os.path.exists('demographics.json'):
        try:
            with open('demographics.json', 'r') as file:
                data = json.load(file)
            return data.get(phone_number, None)
        except (json.JSONDecodeError, IOError):
            print("Error reading demographics.json. Please enter demographics manually.")
    return None

def get_manual_demographics(phone_number):
    demographics = {"phone_number": phone_number} if is_valid_phone_number(phone_number) else {}
    print("Enter demographics (format: DD-MM-YYYY for dates, press Enter to skip):")
    demographics['dob'] = prompt_valid_date("Your Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['father_dob'] = prompt_valid_date("Father's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['mother_dob'] = prompt_valid_date("Mother's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['parents_anniversary'] = prompt_valid_date("Parents' Anniversary (DD-MM-YYYY): ", is_dob=False)
    demographics['spouse_dob'] = prompt_valid_date("Spouse's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['anniversary'] = prompt_valid_date("Your Anniversary (DD-MM-YYYY): ", is_dob=False)
    children_dobs = []
    for i in range(1, 4):
        child_dob = prompt_valid_date(f"Child {i} Date of Birth (DD-MM-YYYY, press Enter to stop): ", is_dob=True)
        if not child_dob:
            break
        children_dobs.append(child_dob)
    if children_dobs:
        demographics['children_dobs'] = children_dobs
    siblings_dobs = []
    for i in range(1, 4):
        sibling_dob = prompt_valid_date(f"Sibling {i} Date of Birth (DD-MM-YYYY, press Enter to stop): ", is_dob=True)
        if not sibling_dob:
            break
        siblings_dobs.append(sibling_dob)
    if siblings_dobs:
        demographics['siblings_dobs'] = siblings_dobs
    while True:
        pincode = input("Your Pincode (6 digits): ").strip()
        if not pincode or is_valid_pincode(pincode):
            demographics['pincode'] = pincode
            break
        print("Invalid pincode. Must be a 6-digit number. Try again or press Enter to skip.")
    return demographics if demographics else None

def extract_all_date_mpins(date):
    """Extracts all 2, 4 digit parts and relevant 4-digit combinations from a date dd-mm-yyyy string."""
    mpins = set()
    if not is_valid_date(date):
        return mpins
    day, month, year = date.split('-')
    day = day.zfill(2)
    month = month.zfill(2)
    year = year.zfill(4)
    year2 = year[-2:]
    items = [day, month, year, year2]
    # All combinations of two parts
    for a, b in permutations(items, 2):
        if len(a + b) == 4:
            mpins.add(a + b)
    # Direct inclusion in case MPIN is "1990" (year itself)
    if len(year) == 4:
        mpins.add(year)
    if len(year2) == 2:
        mpins.add(year2)
    # Add month+day and day+month if not already
    if len(day+month) == 4:
        mpins.add(day+month)
        mpins.add(month+day)
    return mpins

def extract_all_mpins_from_list(values, is_date=True):
    mpins = set()
    if not values: return mpins
    for value in values:
        if is_date:
            mpins.update(extract_all_date_mpins(value))
        else:
            mpins.update(extract_number_4mpins(value))
    return mpins

def extract_number_4mpins(number):
    """Returns set of all 4-digit substrings in a given numeric string."""
    mpins = set()
    if number and number.isdigit():
        for i in range(len(number) - 3):
            mpin_candidate = number[i:i+4]
            if len(mpin_candidate) == 4:
                mpins.add(mpin_candidate)
    return mpins

def generate_demographic_mpins(demographics):
    """Returns a master set of all 4-digit or direct year/2-digit date substrings from all user data."""
    if not demographics:
        return set()
    all_mpins = set()
    # All main date fields
    for field in [
        "dob", "father_dob", "mother_dob",
        "parents_anniversary", "spouse_dob", "anniversary"
    ]:
        val = demographics.get(field)
        if val:
            all_mpins.update(extract_all_date_mpins(val))
    # kids & siblings
    all_mpins.update(extract_all_mpins_from_list(demographics.get("children_dobs", []), is_date=True))
    all_mpins.update(extract_all_mpins_from_list(demographics.get("siblings_dobs", []), is_date=True))
    # Mixed 4-digit pairs using any two date-fields (within siblings/children/parent etc.)
    date_fields = []
    for field in [
        "dob", "father_dob", "mother_dob",
        "parents_anniversary", "spouse_dob", "anniversary"
    ]:
        val = demographics.get(field)
        if val:
            date_fields.append(val)
    date_fields += demographics.get("children_dobs", []) + demographics.get("siblings_dobs", [])
    # All mixed pairs like (day1 + year2), etc.
    for d1, d2 in combinations(date_fields, 2):
        for a in extract_all_date_mpins(d1):
            for b in extract_all_date_mpins(d2):
                if len(a) == 2 and len(b) == 2:
                    all_mpins.add(a+b)
                    all_mpins.add(b+a)
    # pincode, phone number (and last 4 digits)
    if is_valid_pincode(demographics.get("pincode", "")):
        all_mpins.update(extract_number_4mpins(demographics["pincode"]))
    if is_valid_phone_number(demographics.get("phone_number", "")):
        phone = demographics["phone_number"]
        all_mpins.update(extract_number_4mpins(phone))
        all_mpins.add(phone[-4:])  # last 4 digits explicit
        all_mpins.add(phone)       # very conservative: block if PIN = phone
    return all_mpins

def check_mpin_strength(phone_number, mpin):
    if not is_valid_mpin(mpin):
        return "INVALID", [], "Invalid MPIN: Must be a 4-digit number."
    if not is_valid_phone_number(phone_number):
        return "INVALID", [], "Invalid phone number: Must be a 10-digit number."
    common_mpins = load_common_mpins()
    if mpin in common_mpins:
        return "WEAK", ["COMMONLY_USED"], "MPIN is commonly used."
    demographics = load_demographics(phone_number)
    if not demographics:
        print(f"No demographics found for phone number {phone_number}.")
        demographics = get_manual_demographics(phone_number)
        if not demographics:
            return "STRONG", [], "No demographics provided; MPIN not derived from demographics."
    demog_mpins = generate_demographic_mpins(demographics)
    if mpin in demog_mpins:
        return "WEAK", ["DEMOGRAPHIC_MATCH"], "MPIN uses your or your family member's personal data (DOB/pincode/phone, etc)."
    return "STRONG", [], "MPIN is strong."

if __name__ == "__main__":
    phone_number = input("Enter phone number (10 digits): ").strip()
    mpin = input("Enter a 4-digit MPIN: ").strip()
    strength, reasons, message = check_mpin_strength(phone_number, mpin)
    print(f"\nPhone Number: {phone_number}")
    print(f"MPIN: {mpin}")
    print(f"Strength: {strength}")
    print(f"Reasons: {reasons}")
    print(f"Message: {message}")



Phone Number: 7901237648
MPIN: 9293
Strength: WEAK
Reasons: ['DEMOGRAPHIC_MATCH']
Message: MPIN uses your or your family member's personal data (DOB/pincode/phone, etc).
